**import libraries**

In [182]:
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras

tf.__version__

'2.1.0'

In [183]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dhsong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

**load dataset**

In [184]:
raw_train = pd.read_csv('./data/train.csv', encoding='utf-8')
raw_test = pd.read_csv('./data/test.csv', encoding='utf-8')

# Exploring Data

## Basic Traits (shape, columns, missing values, frequency)

### 1. shape

> **Train Data Shape** >> (7613, 5)  
> **Test &nbsp;Data Shape** >> (3263, 4)

In [185]:
raw_train.shape

(7613, 5)

In [186]:
raw_test.shape

(3263, 4)

### 2. columns

> **Train Columns** >> [id, keyword, location, text, <U>target</U>]  
> **Test &nbsp;Columns** >> [id, keyword, location, text]

In [187]:
raw_train.columns

Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')

In [188]:
raw_test.columns

Index(['id', 'keyword', 'location', 'text'], dtype='object')

### 3. missing values

> **keyword** & **location** has <U>missing values</U>

In [189]:
raw_train.isna().sum()

id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [190]:
raw_test.isna().sum()

id             0
keyword       26
location    1105
text           0
dtype: int64

### 4. Frequency

**1. keyword**

> number of **keyword**, which has used **more than 50 times**, is **206** out of 221

In [191]:
keywords = pd.concat([raw_train['keyword'], raw_test['keyword']], ignore_index=True)
keywords.value_counts()

oil%20spill              50
wounds                   50
screaming                50
riot                     50
wildfire                 50
                         ..
forest%20fire            24
threat                   16
inundation               14
radiation%20emergency    14
epicentre                13
Name: keyword, Length: 221, dtype: int64

In [192]:
keywords_50 = keywords.value_counts()[keywords.value_counts() >= 50]
keywords_50.index

Index(['oil%20spill', 'wounds', 'screaming', 'riot', 'wildfire',
       'violent%20storm', 'debris', 'injuries', 'accident',
       'buildings%20burning',
       ...
       'damage', 'destroy', 'rioting', 'suicide%20bombing', 'suicide%20bomb',
       'electrocute', 'fire%20truck', 'first%20responders', 'body%20bags',
       'evacuation'],
      dtype='object', length=206)

**2. location**

> number of **location**, which has used **more than 5 times**, is **174** out of 4521

In [193]:
locations = pd.concat([raw_train['location'], raw_test['location']])
locations.value_counts()

USA                          141
New York                     109
United States                 65
London                        58
Canada                        42
                            ... 
#WashingtonState #Seattle      1
IL                             1
Morris, IL                     1
Pedophilia                     1
Newcastle upon Tyne            1
Name: location, Length: 4521, dtype: int64

In [194]:
locations_20 = locations.value_counts()[locations.value_counts() >= 5]
locations_20.index

Index(['USA', 'New York', 'United States', 'London', 'Canada', 'Nigeria',
       'Worldwide', 'India', 'Los Angeles, CA', 'UK',
       ...
       'Germany', 'Leeds, England', 'Detroit, MI', '??', 'Austin, Texas',
       'World Wide', 'Calgary', 'Anchorage, AK', 'Manchester, England',
       'Asia'],
      dtype='object', length=174)

## Processing Data

### Text

In [195]:
def process_text(dataset):
    dataset['text_proc'] = dataset['text'].str.strip()
    dataset['text_proc'] = dataset['text_proc'].str.lower()

    # Remove <> for labeling
    dataset['text_proc'] = dataset['text_proc'].str.replace('[\<\>]', '')

    # Mention labeling
    dataset['text_proc'] = dataset['text_proc'].str.replace('@(?=[\\w\\d_]+)', ' <mention> ')

    # Hashtag labeling
    dataset['text_proc'] = dataset['text_proc'].str.replace('#(?=[\\w\\d]+)', ' <hashtag> ')
    
    # link labeling
    dataset['text_proc'] = dataset['text_proc'].str.replace('[a-zA-Z]+://[a-zA-Z0-9./]+', ' <link> ')
    
    # remove puntuations
    dataset['text_proc'] = dataset['text_proc'].str.replace('[^a-zA-Z\<\> ]+', '')
    dataset['text_proc'] = dataset['text_proc'].str.replace('[\\s]+', ' ')
    
    # remove stopword
    stop = stopwords.words('english')
    dataset['text_proc'] = dataset['text_proc'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    
    return dataset

In [196]:
train = process_text(raw_train)
test = process_text(raw_test)

In [197]:
train

,id,keyword,location,text,target,text_proc
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason <hashtag> earthquake may allah fo...
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive <hashtag> wildfires evacuation ...
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby <hashtag> alaska smoke <ha...
...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant cranes holding bridge collapse nearb...
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,<mention> ariaahrary <mention> thetawniest con...
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,utckm volcano hawaii <link>
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,police investigating ebike collided car little...


In [198]:
test

,id,keyword,location,text,text_proc
0,0,NaN,NaN,Just happened a terrible car crash,happened terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard <hashtag> earthquake different cities st...
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese fleeing across str...
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting <hashtag> spokane <hashtag...
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills china taiwan
...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,earthquake safety los angeles safety fasteners...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,storm ri worse last hurricane cityampothers ha...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,green line derailment chicago <link>
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,meg issues hazardous weather outlook hwo <link>


In [199]:
words_train = set()
words_test = set()

for tweet in train['text_proc'].str.split():
    for word in tweet:
        words_train.add(word)
for tweet in test['text_proc'].str.split():
    for word in tweet:
        words_test.add(word)

words = words_train | words_test
        
print('# of unique word in train {}'.format(len(words_train)))
print('# of unique word in test {}'.format(len(words_test)))
print('# of unique word only in tset {}'.format(len(words_test - words_train)))
print('# of unique word {}'.format(len(words)))

# of unique word in train 16959
# of unique word in test 10030
# of unique word only in tset 4420
# of unique word 21379


In [200]:
train['tokens'] = train['text_proc'].apply(lambda x: [word for word in x.split() if word in words_train])
test['tokens'] = test['text_proc'].apply(lambda x: [word for word in x.split() if word in words_train])

In [201]:
train

,id,keyword,location,text,target,text_proc,tokens
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,deeds reason <hashtag> earthquake may allah fo...,"[deeds, reason, <hashtag>, earthquake, may, al..."
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,"[forest, fire, near, la, ronge, sask, canada]"
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,residents asked shelter place notified officer...,"[residents, asked, shelter, place, notified, o..."
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,people receive <hashtag> wildfires evacuation ...,"[people, receive, <hashtag>, wildfires, evacua..."
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,got sent photo ruby <hashtag> alaska smoke <ha...,"[got, sent, photo, ruby, <hashtag>, alaska, sm..."
...,...,...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1,two giant cranes holding bridge collapse nearb...,"[two, giant, cranes, holding, bridge, collapse..."
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1,<mention> ariaahrary <mention> thetawniest con...,"[<mention>, ariaahrary, <mention>, thetawniest..."
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1,utckm volcano hawaii <link>,"[utckm, volcano, hawaii, <link>]"
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1,police investigating ebike collided car little...,"[police, investigating, ebike, collided, car, ..."


In [202]:
test

,id,keyword,location,text,text_proc,tokens
0,0,NaN,NaN,Just happened a terrible car crash,happened terrible car crash,"[happened, terrible, car, crash]"
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",heard <hashtag> earthquake different cities st...,"[heard, <hashtag>, earthquake, different, citi..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",forest fire spot pond geese fleeing across str...,"[forest, fire, spot, pond, fleeing, across, st..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting <hashtag> spokane <hashtag...,"[apocalypse, lighting, <hashtag>, spokane, <ha..."
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills china taiwan,"[typhoon, soudelor, kills, china, taiwan]"
...,...,...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,earthquake safety los angeles safety fasteners...,"[earthquake, safety, los, angeles, safety]"
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...,storm ri worse last hurricane cityampothers ha...,"[storm, ri, worse, last, hurricane, cityampoth..."
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...,green line derailment chicago <link>,"[green, line, derailment, chicago, <link>]"
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...,meg issues hazardous weather outlook hwo <link>,"[meg, issues, hazardous, weather, outlook, hwo..."


In [203]:
x_train_tokens = train['tokens']
y_train_target = train['target']
x_test_tokens = test['tokens']

In [204]:
x_train_tokens.shape

(7613,)

In [205]:
y_train_target.shape

(7613,)

In [206]:
x_test_tokens.shape

(3263,)

In [207]:
tokens = ['<pad>'] + sorted(words_train)
tokens

['<pad>',
 '<hashtag>',
 '<link>',
 '<mention>',
 'aa',
 'aaaa',
 'aaaaaaallll',
 'aaarrrgghhh',
 'aaceorg',
 'aal',
 'aampb',
 'aampw',
 'aan',
 'aannnnd',
 'aar',
 'aaronthefm',
 'aashiqui',
 'ab',
 'aba',
 'abandon',
 'abandoned',
 'abandonedpics',
 'abandoning',
 'abbandoned',
 'abbott',
 'abbruchsimulator',
 'abbswinston',
 'abbyairshow',
 'abc',
 'abcchicago',
 'abceyewitness',
 'abcnews',
 'abcnorio',
 'abcs',
 'abe',
 'aberdeen',
 'aberdeenfanpage',
 'aberdeenfc',
 'aberystwythshrewsbury',
 'abes',
 'abetter',
 'abha',
 'abia',
 'ability',
 'abject',
 'ablaze',
 'able',
 'ableg',
 'abninfvet',
 'aboard',
 'abomb',
 'abombed',
 'abomination',
 'abortion',
 'abortions',
 'abouts',
 'abovewould',
 'abrancaballero',
 'abs',
 'absence',
 'absolute',
 'absolutely',
 'absolutsumya',
 'abstorm',
 'abstract',
 'absurd',
 'absurdly',
 'abubaraa',
 'abuse',
 'abused',
 'abuseddesolateamplost',
 'abuses',
 'abusing',
 'abysmaljoiner',
 'ac',
 'acaciapenn',
 'academia',
 'acarewornheart',
 

In [208]:
token2idx = {token: idx for idx, token in enumerate(tokens)}
idx2token = {idx: token for idx, token in enumerate(tokens)}
print(len(idx2token))
idx2token

16960


{0: '<pad>',
 1: '<hashtag>',
 2: '<link>',
 3: '<mention>',
 4: 'aa',
 5: 'aaaa',
 6: 'aaaaaaallll',
 7: 'aaarrrgghhh',
 8: 'aaceorg',
 9: 'aal',
 10: 'aampb',
 11: 'aampw',
 12: 'aan',
 13: 'aannnnd',
 14: 'aar',
 15: 'aaronthefm',
 16: 'aashiqui',
 17: 'ab',
 18: 'aba',
 19: 'abandon',
 20: 'abandoned',
 21: 'abandonedpics',
 22: 'abandoning',
 23: 'abbandoned',
 24: 'abbott',
 25: 'abbruchsimulator',
 26: 'abbswinston',
 27: 'abbyairshow',
 28: 'abc',
 29: 'abcchicago',
 30: 'abceyewitness',
 31: 'abcnews',
 32: 'abcnorio',
 33: 'abcs',
 34: 'abe',
 35: 'aberdeen',
 36: 'aberdeenfanpage',
 37: 'aberdeenfc',
 38: 'aberystwythshrewsbury',
 39: 'abes',
 40: 'abetter',
 41: 'abha',
 42: 'abia',
 43: 'ability',
 44: 'abject',
 45: 'ablaze',
 46: 'able',
 47: 'ableg',
 48: 'abninfvet',
 49: 'aboard',
 50: 'abomb',
 51: 'abombed',
 52: 'abomination',
 53: 'abortion',
 54: 'abortions',
 55: 'abouts',
 56: 'abovewould',
 57: 'abrancaballero',
 58: 'abs',
 59: 'absence',
 60: 'absolute',
 61

In [211]:
max_length = 0
for tweet in x_train_tokens:
    if len(tweet) > max_length:
        max_length = len(tweet)
        print('[{}] {}'.format(max_length, tweet))

[8] ['deeds', 'reason', '<hashtag>', 'earthquake', 'may', 'allah', 'forgive', 'us']
[11] ['residents', 'asked', 'shelter', 'place', 'notified', 'officers', 'evacuation', 'shelter', 'place', 'orders', 'expected']
[15] ['<hashtag>', 'rockyfire', 'update', 'california', 'hwy', 'closed', 'directions', 'due', 'lake', 'county', 'fire', '<hashtag>', 'cafire', '<hashtag>', 'wildfires']
[16] ['haha', 'south', 'tampa', 'getting', 'flooded', 'hah', 'wait', 'second', 'live', 'south', 'tampa', 'gonna', 'gonna', 'fvck', '<hashtag>', 'flooding']
[17] ['rt', 'naayf', 'first', 'accident', 'years', 'turning', 'onto', 'chandanee', 'magu', 'near', 'mma', 'taxi', 'rammed', 'halfway', 'turned', 'everyone', 'conf']
[19] ['accident', 'left', 'lane', 'blocked', '<hashtag>', 'manchester', 'rt', 'nb', 'eddy', 'rd', 'stop', 'go', 'traffic', 'back', 'nha', 'delay', 'mins', '<hashtag>', 'traffic']
[22] ['anyone', 'need', 'pu', 'tonight', 'play', 'hybrid', 'slayer', 'ps', 'eu', 'hmu', '<mention>', 'codsandscrims', '

In [212]:
max_length = 0
for tweet in x_test_tokens:
    if len(tweet) > max_length:
        max_length = len(tweet)
        print('[{}] {}'.format(max_length, tweet))

[4] ['happened', 'terrible', 'car', 'crash']
[8] ['heard', '<hashtag>', 'earthquake', 'different', 'cities', 'stay', 'safe', 'everyone']
[9] ['forest', 'fire', 'spot', 'pond', 'fleeing', 'across', 'street', 'cannot', 'save']
[12] ['birmingham', 'wholesale', 'market', 'ablaze', 'bbc', 'news', 'fire', 'breaks', 'birminghams', 'wholesale', 'market', '<link>']
[15] ['diss', 'song', 'people', 'take', 'thing', 'run', 'smh', 'eye', 'opener', 'though', 'set', 'game', 'ablaze', '<mention>', 'cyhitheprynce']
[19] ['ir', 'icemoon', 'aftershock', '<link>', '<mention>', 'djicemoon', '<hashtag>', 'dubstep', '<hashtag>', 'trapmusic', '<hashtag>', 'dnb', '<hashtag>', 'edm', '<hashtag>', 'dance', '<hashtag>', 'ices', '<link>']
[27] ['hundreds', 'migrants', 'feared', 'drowned', 'libya', '<link>', '<hashtag>', 'news', '<hashtag>', 'bbc', '<hashtag>', 'cnn', '<hashtag>', 'msnbc', '<hashtag>', 'nyt', '<hashtag>', 'tcot', '<hashtag>', '<hashtag>', '<hashtag>', 'ccot', '<hashtag>', '<hashtag>', 'p', '<hashta

In [213]:
sequence_length = 30

In [218]:
x_train = list(map(lambda tweet: [token2idx[token] for token in tweet], x_train_tokens))
x_test = list(map(lambda tweet: [token2idx[token] for token in tweet], x_test_tokens))

In [222]:
x_train = keras.preprocessing.sequence.pad_sequences(
    x_train, 
    maxlen=sequence_length, 
    padding='post', 
    truncating='post')
x_test = keras.preprocessing.sequence.pad_sequences(
    x_test, 
    maxlen=sequence_length, 
    padding='post', 
    truncating='post')

In [286]:
x_train.shape

(7613, 30)

In [287]:
x_test.shape

(3263, 30)

In [288]:
y_train = np.array(y_train_target)

### modeling

In [297]:
class TweetModel(keras.Model):
    def __init__(self, vocab_size, embedding_dim):
        super(TweetModel, self).__init__()
        
        self.embedding = keras.layers.Embedding(vocab_size, embedding_dim)
        
        self.lstm1 = keras.layers.LSTM(512, return_sequences=True)
        self.dist1 = keras.layers.TimeDistributed(keras.layers.Dropout(rate=0.5))
        
        self.lstm2 = keras.layers.LSTM(256)
        self.drop2 = keras.layers.Dropout(rate = 0.25)
    
        self.dense3 = keras.layers.Dense(units=128, activation='relu')
        self.drop3 = keras.layers.Dropout(rate = 0.2)

        self.dense4 = keras.layers.Dense(units=1, activation='sigmoid')
        
    def call(self, x, training=False):
        x = self.embedding(x)
        
        layer1 = self.lstm1(x)
        layer1 = self.dist1(layer1)
        
        layer2 = self.lstm2(layer1)
        layer2 = self.drop2(layer2)

        layer3 = self.dense3(layer2)
        layer3 = self.drop3(layer3)

        y = self.dense4(layer3)
        
        return y

In [298]:
model = TweetModel(len(tokens), sequence_length)

In [299]:
learning_rate = 0.01
epochs = 100
batch_size = 128
iterations = x_train.shape[0] // batch_size

In [300]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(10000)
dataset = dataset.prefetch(10000)
dataset = dataset.batch(batch_size)
dataset = dataset.repeat()

dataset_iterator = iter(dataset)

In [301]:
def loss_fn(model, features, labels, training=True):
    y = model(features, training)
    labels = tf.cast(labels, tf.float32)
    loss = -tf.reduce_mean(labels * tf.math.log(y) + (1 - labels) * tf.math.log(1 - y))
    return loss

In [302]:
def accuracy_fn(model, features, labels, training=False):
    y = model(features, training)
    prediction = tf.cast(y > 0.5, tf.int64)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, labels), tf.float32))
    return accuracy

In [303]:
optimizer = keras.optimizers.Adam(learning_rate)

In [304]:
for epoch in range(epochs):
    train_loss = 0.
    train_acc = 0.
    for iteration in range(iterations):
        train_features, train_labels = dataset_iterator.get_next()
        with tf.GradientTape() as tape:
            loss = loss_fn(model, train_features, train_labels, True)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        
        train_loss += loss
        
        acc = accuracy_fn(model, train_features, train_labels, False)
        train_acc += acc
        
        if iteration % 5 == 0:
            print('Epoch {:3d} [{:4d}/{:4d}] loss={:7.4f} acc={:.2%}'.format(epoch + 1, iteration + 1, iterations, loss, acc))
    
    train_loss /= iterations
    train_acc /= iterations
    
    val_features, val_labels = dataset_iterator.get_next()
    val_loss = loss_fn(model, val_features, val_labels, False)
    val_acc = accuracy_fn(model, val_features, val_labels, False)

    print('*** Epoch {:3d} TRAINING loss={:7.4f} acc={:.2%} VALIDATION loss={:7.4f} acc={:.2%} ***\n'.format(epoch + 1, train_loss, train_acc, val_loss, val_acc))
            

Epoch   1 [   1/  59] loss= 0.6933 acc=57.81%
Epoch   1 [   6/  59] loss= 2.4677 acc=48.83%
Epoch   1 [  11/  59] loss= 0.6871 acc=57.81%
Epoch   1 [  16/  59] loss= 0.6879 acc=56.25%
Epoch   1 [  21/  59] loss= 0.6866 acc=56.25%
Epoch   1 [  26/  59] loss= 0.6867 acc=56.25%
Epoch   1 [  31/  59] loss= 0.7020 acc=51.56%
Epoch   1 [  36/  59] loss= 0.6955 acc=53.12%
Epoch   1 [  41/  59] loss= 0.6866 acc=56.25%
Epoch   1 [  46/  59] loss= 0.6834 acc=57.81%
Epoch   1 [  51/  59] loss= 0.6817 acc=57.81%
Epoch   1 [  56/  59] loss= 0.6957 acc=53.91%
*** Epoch   1 TRAINING loss= 0.7389 acc=56.52% VALIDATION loss= 0.6381 acc=70.49% ***

Epoch   2 [   1/  59] loss= 0.6899 acc=55.47%
Epoch   2 [   6/  59] loss= 0.6792 acc=58.59%
Epoch   2 [  11/  59] loss= 0.6929 acc=53.91%
Epoch   2 [  16/  59] loss= 0.7028 acc=49.22%
Epoch   2 [  21/  59] loss= 0.6779 acc=60.16%
Epoch   2 [  26/  59] loss= 0.6841 acc=57.03%
Epoch   2 [  31/  59] loss= 0.6789 acc=58.59%
Epoch   2 [  36/  59] loss= 0.6906 acc=

*** Epoch  13 TRAINING loss= 0.6833 acc=57.04% VALIDATION loss= 0.6870 acc=55.74% ***

Epoch  14 [   1/  59] loss= 0.6968 acc=52.34%
Epoch  14 [   6/  59] loss= 0.6940 acc=53.12%
Epoch  14 [  11/  59] loss= 0.6791 acc=58.59%
Epoch  14 [  16/  59] loss= 0.6813 acc=57.81%
Epoch  14 [  21/  59] loss= 0.6649 acc=64.06%
Epoch  14 [  26/  59] loss= 0.6833 acc=57.03%
Epoch  14 [  31/  59] loss= 0.6747 acc=60.16%
Epoch  14 [  36/  59] loss= 0.6922 acc=53.91%
Epoch  14 [  41/  59] loss= 0.7038 acc=50.00%
Epoch  14 [  46/  59] loss= 0.6720 acc=60.94%
Epoch  14 [  51/  59] loss= 0.6544 acc=67.19%
Epoch  14 [  56/  59] loss= 0.6787 acc=58.59%
*** Epoch  14 TRAINING loss= 0.6831 acc=57.12% VALIDATION loss= 0.7163 acc=45.90% ***

Epoch  15 [   1/  59] loss= 0.6949 acc=53.12%
Epoch  15 [   6/  59] loss= 0.6741 acc=60.16%
Epoch  15 [  11/  59] loss= 0.6811 acc=57.81%
Epoch  15 [  16/  59] loss= 0.6978 acc=51.56%
Epoch  15 [  21/  59] loss= 0.6952 acc=52.34%
Epoch  15 [  26/  59] loss= 0.6776 acc=59.38

Epoch  26 [  51/  59] loss= 0.6541 acc=67.97%
Epoch  26 [  56/  59] loss= 0.7040 acc=49.22%
*** Epoch  26 TRAINING loss= 0.6832 acc=57.08% VALIDATION loss= 0.6995 acc=50.82% ***

Epoch  27 [   1/  59] loss= 0.6812 acc=57.81%
Epoch  27 [   6/  59] loss= 0.6748 acc=60.16%
Epoch  27 [  11/  59] loss= 0.6899 acc=54.69%
Epoch  27 [  16/  59] loss= 0.6900 acc=54.69%
Epoch  27 [  21/  59] loss= 0.6832 acc=57.03%
Epoch  27 [  26/  59] loss= 0.6991 acc=51.56%
Epoch  27 [  31/  59] loss= 0.6658 acc=63.28%
Epoch  27 [  36/  59] loss= 0.6959 acc=52.34%
Epoch  27 [  41/  59] loss= 0.6581 acc=66.41%
Epoch  27 [  46/  59] loss= 0.7028 acc=50.00%
Epoch  27 [  51/  59] loss= 0.6790 acc=58.59%
Epoch  27 [  56/  59] loss= 0.6726 acc=60.94%
*** Epoch  27 TRAINING loss= 0.6832 acc=57.08% VALIDATION loss= 0.7006 acc=50.82% ***

Epoch  28 [   1/  59] loss= 0.6832 acc=57.03%
Epoch  28 [   6/  59] loss= 0.6719 acc=60.94%
Epoch  28 [  11/  59] loss= 0.6648 acc=63.28%
Epoch  28 [  16/  59] loss= 0.6902 acc=54.69

Epoch  39 [  41/  59] loss= 0.6809 acc=57.81%
Epoch  39 [  46/  59] loss= 0.6809 acc=57.81%
Epoch  39 [  51/  59] loss= 0.6809 acc=57.81%
Epoch  39 [  56/  59] loss= 0.6902 acc=54.69%
*** Epoch  39 TRAINING loss= 0.6834 acc=57.03% VALIDATION loss= 0.6822 acc=57.38% ***

Epoch  40 [   1/  59] loss= 0.6787 acc=58.59%
Epoch  40 [   6/  59] loss= 0.6746 acc=60.16%
Epoch  40 [  11/  59] loss= 0.6833 acc=57.03%
Epoch  40 [  16/  59] loss= 0.6672 acc=63.28%
Epoch  40 [  21/  59] loss= 0.6773 acc=59.38%
Epoch  40 [  26/  59] loss= 0.6770 acc=59.38%
Epoch  40 [  31/  59] loss= 0.6876 acc=55.47%
Epoch  40 [  36/  59] loss= 0.6721 acc=60.94%
Epoch  40 [  41/  59] loss= 0.6626 acc=64.06%
Epoch  40 [  46/  59] loss= 0.6833 acc=57.03%
Epoch  40 [  51/  59] loss= 0.6737 acc=60.16%
Epoch  40 [  56/  59] loss= 0.6785 acc=58.59%
*** Epoch  40 TRAINING loss= 0.6832 acc=57.08% VALIDATION loss= 0.7019 acc=50.82% ***

Epoch  41 [   1/  59] loss= 0.6762 acc=59.38%
Epoch  41 [   6/  59] loss= 0.6786 acc=58.59

Epoch  52 [  31/  59] loss= 0.6698 acc=61.72%
Epoch  52 [  36/  59] loss= 0.6832 acc=57.03%
Epoch  52 [  41/  59] loss= 0.6628 acc=64.06%
Epoch  52 [  46/  59] loss= 0.7064 acc=49.22%
Epoch  52 [  51/  59] loss= 0.6901 acc=54.69%
Epoch  52 [  56/  59] loss= 0.6743 acc=60.16%
*** Epoch  52 TRAINING loss= 0.6832 acc=57.10% VALIDATION loss= 0.7057 acc=49.18% ***

Epoch  53 [   1/  59] loss= 0.6989 acc=51.56%
Epoch  53 [   6/  59] loss= 0.6900 acc=54.69%
Epoch  53 [  11/  59] loss= 0.6764 acc=59.38%
Epoch  53 [  16/  59] loss= 0.6855 acc=56.25%
Epoch  53 [  21/  59] loss= 0.6787 acc=58.59%
Epoch  53 [  26/  59] loss= 0.6968 acc=52.34%
Epoch  53 [  31/  59] loss= 0.6832 acc=57.03%
Epoch  53 [  36/  59] loss= 0.6676 acc=62.50%
Epoch  53 [  41/  59] loss= 0.6764 acc=59.38%
Epoch  53 [  46/  59] loss= 0.6854 acc=56.25%
Epoch  53 [  51/  59] loss= 0.6874 acc=55.47%
Epoch  53 [  56/  59] loss= 0.6752 acc=60.16%
*** Epoch  53 TRAINING loss= 0.6832 acc=57.07% VALIDATION loss= 0.6954 acc=52.46% ***

Epoch  65 [  21/  59] loss= 0.6772 acc=59.38%
Epoch  65 [  26/  59] loss= 0.6771 acc=59.38%
Epoch  65 [  31/  59] loss= 0.6689 acc=62.50%
Epoch  65 [  36/  59] loss= 0.6833 acc=57.03%
Epoch  65 [  41/  59] loss= 0.6635 acc=64.06%
Epoch  65 [  46/  59] loss= 0.7107 acc=47.66%
Epoch  65 [  51/  59] loss= 0.6946 acc=53.12%
Epoch  65 [  56/  59] loss= 0.6968 acc=52.34%
*** Epoch  65 TRAINING loss= 0.6834 acc=56.99% VALIDATION loss= 0.6680 acc=62.30% ***

Epoch  66 [   1/  59] loss= 0.7036 acc=50.00%
Epoch  66 [   6/  59] loss= 0.6676 acc=62.50%
Epoch  66 [  11/  59] loss= 0.6677 acc=62.50%
Epoch  66 [  16/  59] loss= 0.6676 acc=62.50%
Epoch  66 [  21/  59] loss= 0.6900 acc=54.69%
Epoch  66 [  26/  59] loss= 0.6787 acc=58.59%
Epoch  66 [  31/  59] loss= 0.6715 acc=60.94%
Epoch  66 [  36/  59] loss= 0.6736 acc=60.16%
Epoch  66 [  41/  59] loss= 0.6834 acc=57.03%
Epoch  66 [  46/  59] loss= 0.6834 acc=57.03%
Epoch  66 [  51/  59] loss= 0.6646 acc=63.28%
Epoch  66 [  56/  59] loss= 0.7036 acc=

Epoch  78 [  11/  59] loss= 0.6898 acc=54.69%
Epoch  78 [  16/  59] loss= 0.6678 acc=62.50%
Epoch  78 [  21/  59] loss= 0.6877 acc=55.47%
Epoch  78 [  26/  59] loss= 0.6786 acc=58.59%
Epoch  78 [  31/  59] loss= 0.6856 acc=56.25%
Epoch  78 [  36/  59] loss= 0.6717 acc=60.94%
Epoch  78 [  41/  59] loss= 0.6809 acc=57.81%
Epoch  78 [  46/  59] loss= 0.7026 acc=50.78%
Epoch  78 [  51/  59] loss= 0.6976 acc=52.34%
Epoch  78 [  56/  59] loss= 0.7040 acc=50.00%
*** Epoch  78 TRAINING loss= 0.6833 acc=57.04% VALIDATION loss= 0.6869 acc=55.74% ***

Epoch  79 [   1/  59] loss= 0.6987 acc=51.56%
Epoch  79 [   6/  59] loss= 0.6854 acc=56.25%
Epoch  79 [  11/  59] loss= 0.6609 acc=65.62%
Epoch  79 [  16/  59] loss= 0.6894 acc=54.69%
Epoch  79 [  21/  59] loss= 0.6874 acc=55.47%
Epoch  79 [  26/  59] loss= 0.6709 acc=61.72%
Epoch  79 [  31/  59] loss= 0.6812 acc=57.81%
Epoch  79 [  36/  59] loss= 0.6916 acc=53.91%
Epoch  79 [  41/  59] loss= 0.6975 acc=51.56%
Epoch  79 [  46/  59] loss= 0.6771 acc=

Epoch  91 [   1/  59] loss= 0.6833 acc=57.03%
Epoch  91 [   6/  59] loss= 0.7016 acc=50.00%
Epoch  91 [  11/  59] loss= 0.6914 acc=53.91%
Epoch  91 [  16/  59] loss= 0.6978 acc=51.56%
Epoch  91 [  21/  59] loss= 0.7083 acc=47.66%
Epoch  91 [  26/  59] loss= 0.6812 acc=57.81%
Epoch  91 [  31/  59] loss= 0.6999 acc=50.78%
Epoch  91 [  36/  59] loss= 0.6789 acc=58.59%
Epoch  91 [  41/  59] loss= 0.6878 acc=55.47%
Epoch  91 [  46/  59] loss= 0.6762 acc=59.38%
Epoch  91 [  51/  59] loss= 0.6880 acc=55.47%
Epoch  91 [  56/  59] loss= 0.6786 acc=58.59%
*** Epoch  91 TRAINING loss= 0.6831 acc=57.12% VALIDATION loss= 0.7163 acc=45.90% ***

Epoch  92 [   1/  59] loss= 0.6925 acc=53.91%
Epoch  92 [   6/  59] loss= 0.6898 acc=54.69%
Epoch  92 [  11/  59] loss= 0.6811 acc=57.81%
Epoch  92 [  16/  59] loss= 0.6897 acc=54.69%
Epoch  92 [  21/  59] loss= 0.6724 acc=60.94%
Epoch  92 [  26/  59] loss= 0.6875 acc=55.47%
Epoch  92 [  31/  59] loss= 0.6832 acc=57.03%
Epoch  92 [  36/  59] loss= 0.7085 acc=

### evaluation

In [307]:
acc = accuracy_fn(model, x_train, y_train, False)
print('Accuracy : {:.2%}'.format(acc))

Accuracy : 57.03%
